In [10]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [33]:
from platform import python_version
print(python_version())

3.8.8


### Recuperando datos

In [11]:
# Reclamos
file_name = 'data/pre_processed_data.csv'
datos_reclamos = pd.read_pickle(file_name)

# Catalogos BN
catalogo_bn_motivo = pd.read_csv('catalogo/catalogo_motivos.csv')
catalogo_bn_motivo['Cod Motivo'] = catalogo_bn_motivo['Cod Motivo'].astype(str)

catalogo_bn_servicio = pd.read_csv('catalogo/catalogo_servicios.csv')
catalogo_bn_servicio['Cod Servicio'] = catalogo_bn_servicio['Cod Servicio'].astype(str)

# Catalogos SBS
catalogo_sbs_motivo = pd.read_csv('catalogo/catalogo_sbs_motivo.csv')
catalogo_sbs_servicio = pd.read_csv('catalogo/catalogo_sbs_motivo.csv')

#### Recuperando codigos SBS

In [14]:
# Merge Motivos
datos_reclamos_merged = pd.merge(
                            datos_reclamos,
                            catalogo_bn_motivo[['Cod Motivo', 'Cod Motivo SBS', 'Motivo Corregido']],
                            how="left",
                            left_on='Cod Motivo',
                            right_on='Cod Motivo',
                            sort=True,
                            suffixes=("_x", "_y"),
                        )
# Merge servicios
datos_reclamos_merged = pd.merge(
                            datos_reclamos_merged,
                            catalogo_bn_servicio[['Cod Servicio', 'Cod Servicio SBS', 'Servicio Corregido']],
                            how="left",
                            left_on='Cod Servicio',
                            right_on='Cod Servicio',
                            sort=True,
                            suffixes=("_x", "_y"),
                        )

datos_reclamos_merged['Cod Servicio SBS'] = datos_reclamos_merged['Cod Servicio SBS'].fillna(-1).astype(int)
datos_reclamos_merged['Cod Motivo SBS'] = datos_reclamos_merged['Cod Motivo SBS'].fillna(-1).astype(int)


In [15]:
len(datos_reclamos), len(datos_reclamos_merged)

(248058, 248058)

#### Recuperando detalles SBS

In [23]:
catalogo_sbs_motivo = catalogo_sbs_motivo.set_index('Cod SBS').T.to_dict('dict')
catalogo_sbs_servicio = catalogo_sbs_servicio.set_index('Cod SBS').T.to_dict('dict')

<ipython-input-23-34b059a8c9d0>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  catalogo_sbs_motivo = catalogo_sbs_motivo.set_index('Cod SBS').T.to_dict('dict')
<ipython-input-23-34b059a8c9d0>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  catalogo_sbs_servicio = catalogo_sbs_servicio.set_index('Cod SBS').T.to_dict('dict')


In [31]:
def nivel_reclamo(x,y):
    try:
        nivel = catalogo_sbs_motivo[x]['Nivel']
    except:
        try:
            nivel = catalogo_sbs_servicio[y]['Nivel']
        except:
            nivel = None
    return nivel

def tipo_reclamo(x,y):
    try:
        tipo = catalogo_sbs_motivo[x]['Tipo']
    except:
        try:
            tipo = catalogo_sbs_servicio[y]['Tipo']
        except:
            tipo = None
    return tipo

datos_reclamos_merged['Nivel'] = datos_reclamos_merged.apply(lambda x: nivel_reclamo(x['Cod Motivo SBS'], x['Cod Servicio SBS']), axis=1)
datos_reclamos_merged['Tipo'] = datos_reclamos_merged.apply(lambda x: tipo_reclamo(x['Cod Motivo SBS'], x['Cod Servicio SBS']), axis=1)

In [29]:
datos_reclamos_merged.head()

,N° Reclamo,Nombre,N° Docum.,Servicio,Cod Motivo,Motivo,Cod. Atm,Det. AtmError,N° Cajero,Monto,...,fecha_fase6,fecha_fase7,Cod Servicio,Cod Motivo SBS,Motivo Corregido,Cod Servicio SBS,Servicio Corregido,nivel,Nivel,Tipo
0,023818-7,TORRES SENMACHE PATHY PAMELA,41114765,CUENTA DE AHORRO,10000,ICCC - INCUMPLIMIENTO DE CLAUSULAS DEL CONTRAT...,NaN,NaN,NaN,NaN,...,2018-10-03,2018-10-03,100,1,"INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, ...",102,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,1er,1er,1er
1,002718-4016,CARLOS RIQUELME ROMAN,43471167,CUENTA DE AHORRO,10000,ICCC - INCUMPLIMIENTO DE CLAUSULAS DEL CONTRAT...,NaN,NaN,NaN,NaN,...,2018-09-26,2018-09-26,100,1,"INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, ...",102,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,1er,1er,1er
2,002719-6276,RICARDO ARTURO RIVERA NUÑEZ,30831857,CUENTA DE AHORRO,10000,ICCC - INCUMPLIMIENTO DE CLAUSULAS DEL CONTRAT...,NaN,NaN,NaN,892.74,...,2020-02-10,2020-02-11,100,1,"INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, ...",102,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,1er,1er,1er
3,010119-356,VERAMENDI ANCASI JAVIER,09952556,CUENTA DE AHORRO,10000,ICCC - INCUMPLIMIENTO DE CLAUSULAS DEL CONTRAT...,NaN,NaN,NaN,8000,...,2019-12-20,2019-12-20,100,1,"INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, ...",102,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,1er,1er,1er
4,010119-354,MEDINA RAMOS JOSE LUIS,29353518,CUENTA DE AHORRO,10000,ICCC - INCUMPLIMIENTO DE CLAUSULAS DEL CONTRAT...,NaN,NaN,NaN,NaN,...,2019-11-12,2019-11-12,100,1,"INCUMPLIMIENTO DE CLAUSULAS DE LOS CONTRATOS, ...",102,CUENTA DE AHORRO (DESTINADA A DEPÓSITO DE REMU...,1er,1er,1er


### Saving data

In [ ]:
object_df.to_csv('data/pre_processed_sbs.csv', index=False)  